# Filtering Queries

There are many ways that you can query a subset or subgraph of the data in the graph.

In this module, you will:

- Review basic query filtering.

- Evaluate strings in your query filtering.

- Retrieve data using patterns in the graph.

- Learn how to specify multiple or optional MATCH clauses.

## Filtering with WHERE

### 1. Testing Equality

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
AND m.year = 2013
RETURN m.title
```

### 2. Testing Inequality

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name <> 'Tom Hanks'
AND m.title = 'Captain Phillips'
RETURN p.name
```

### 3. Testing less than or greater than

```cypher
MATCH (m:Movie) WHERE m.title = 'Toy Story'
RETURN
    m.year < 1995 AS lessThan, //  Less than (false)
    m.year <= 1995 AS lessThanOrEqual, // Less than or equal(true)
    m.year > 1995 AS moreThan, // More than (false)
    m.year >= 1995 AS moreThanOrEqual // More than or equal (true)
```

### 4. Testing Ranges

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
AND  2005 <= m.year <= 2010
RETURN m.title, m.released
```

### 5. Testing null property values

```cypher
MATCH (p:Person)
WHERE p.died IS NOT NULL
AND p.born.year >= 1985
RETURN p.name, p.born, p.died
```

And we can test if a property exists using the IS NULL predicate:

```cypher
MATCH (p:Person)
WHERE p.died IS NULL
AND p.born.year <= 1922
RETURN p.name, p.born, p.died
```

### 6. Testing labels or patterns?

```cypher
MATCH (p:Person)
WHERE  p.born.year > 1960
AND p:Actor
AND p:Director
RETURN p.name, p.born, labels(p)
```

```cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(p)
WHERE  p.born.year > 1960
RETURN p.name, p.born, labels(p), m.title
````

### 7. Testing list inclusion

```cypher
MATCH (m:Movie)
WHERE "Israel" IN m.countries
RETURN m.title, m.languages, m.countries
```

```
MATCH (d:Director)-[:DIRECTED]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE m.year = 2000 AND g.name = "Horror"
RETURN d.name
```

### 8. Testing strings

When the property is a string type, you can filter by the starting characters in the string:

```cypher
MATCH (m:Movie)
WHERE  m.title STARTS WITH 'Toy Story'
RETURN m.title, m.released
```

And you can filter queries whose properties end with a set of characters:

```cypher
MATCH (m:Movie)
WHERE  m.title ENDS WITH ' I'
RETURN m.title, m.released
```

Additionally, you can test if a substring is contained in a property:

```cypher
MATCH (m:Movie)
WHERE  m.title CONTAINS 'River'
RETURN m.title, m.released
```

Case-sensitive strings

```cypher
MATCH (p:Person)
WHERE toLower(p.name) ENDS WITH 'demille'
RETURN p.name
```

Conversely, we can do the same test with upper-case:

```cypher
MATCH (p:Person)
WHERE toUpper(p.name) ENDS WITH 'DEMILLE'
RETURN p.name
```

### 8. About indexes for queries

If you transform a string property during a query, such as `toUpper()` or `toLower()`, the query engine turns off the use of the index.

With any query, you can always check if an index will be used by prefixing the query with `EXPLAIN`.

```cypher
EXPLAIN MATCH (m:Movie)
WHERE  m.title STARTS WITH 'Toy Story'
RETURN m.title, m.released
```